# Inclass Submission: Use the code below and try to build multipe ANN models using combinations of hyper parameters, Optimisers and Activation function/Layers

Please use seed value of 42 to be consistent with the lead instructor. Model 1 and Model 2 has been done for you. Feel free to choose your own combination  - 5 marks

In [1]:
#!pip install pydot

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Read in the insurance dataset
insurance = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")

In [3]:
insurance.shape

(1338, 7)

In [4]:
# Check out the data
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
# Turn all categories into numbers
insurance_one_hot = pd.get_dummies(insurance)
insurance_one_hot.head() # view the converted columns
# Create X & y values
X = insurance_one_hot.drop("charges", axis=1)
y = insurance_one_hot["charges"]

In [6]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# Create training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42) # set random state for reproducible splits

# Create column transformer (this will help us normalize/preprocess our data)
ct = make_column_transformer(
    (MinMaxScaler(), ["age", "bmi", "children"]), # get all values between 0 and 1
    (OneHotEncoder(handle_unknown="ignore"), ["sex", "smoker", "region"])
)

# Create X & y
X = insurance.drop("charges", axis=1)
y = insurance["charges"]

# Build our train and test sets (use random state to ensure same split as before)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit column transformer on the training data only (doing so on test data would result in data leakage)
ct.fit(X_train)

# Transform training and test data with normalization (MinMaxScalar) and one hot encoding (OneHotEncoder)
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [7]:
# Non-normalized and non-one-hot encoded data example
X_train.loc[0]

age                19
sex            female
bmi              27.9
children            0
smoker            yes
region      southwest
Name: 0, dtype: object

In [8]:
# Normalized and one-hot encoded example
X_train_normal[0]

array([0.60869565, 0.10734463, 0.4       , 1.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        ])

In [9]:
# Notice the normalized/one-hot encoded shape is larger because of the extra columns
X_train_normal.shape, X_train.shape

((1070, 11), (1070, 6))

In [10]:
# link for Optimisers https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

# Model 1 --> Use SGD with no momemtum

In [11]:
# Set random seed
tf.random.set_seed(42)

# Build the model (3 layers, 100, 10,1 units)
insurance_model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
insurance_model_1.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0),
                          metrics=['mae'])

# Fit the model for 200 epochs (call it model 1)
insurance_model_1.fit(X_train_normal, y_train, epochs=200, verbose=0) 

In [12]:
insurance_model_1_loss, insurance_model_1_mae = insurance_model_1.evaluate(X_test_normal, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 3486.6892 - mae: 3486.6892


In [13]:
# Check the model summary
insurance_model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1200      
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2,221
Trainable params: 2,221
Non-trainable params: 0
_________________________________________________________________


# Model 2 --> Use Adam

In [14]:
# Set random seed
tf.random.set_seed(42)

# Build the model (3 layers, 100, 10, 1 units)
insurance_model_2 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
 
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
insurance_model_2.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mae'])

# Fit the model for 200 epochs (same as insurance_model_1)
insurance_model_2.fit(X_train_normal, y_train, epochs=200, verbose=0) 

In [15]:
# Evaulate 2nd model
insurance_model_2_loss, insurance_model_2_mae = insurance_model_2.evaluate(X_test_normal, y_test)

9/9 [==============================] - 0s 769us/step - loss: 3171.5774 - mae: 3171.5774


# Model 3 --> Add another hidden layer with 100 neurons

In [18]:
# Set random seed
tf.random.set_seed(42)

insurance_model_3 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
insurance_model_3.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mae'])

# Fit the model for 200 epochs (same as insurance_model_1)
insurance_model_3.fit(X_train_normal, y_train, epochs=200, verbose=0)

In [19]:
# Evaulate 3rd model
insurance_model_3_loss, insurance_model_3_mae = insurance_model_3.evaluate(X_test_normal, y_test)

9/9 [==============================] - 0s 3ms/step - loss: 3175.5801 - mae: 3175.5801


# Model 4 --> SGD with momemtum=0.9  + Back to 3 layers + Change learning rate= 0.001

In [20]:
# Set random seed
tf.random.set_seed(42)

insurance_model_4 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),

  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
insurance_model_4.compile(loss=tf.keras.losses.mae, optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum =0.9), metrics=['mae'])

# Fit the model for 200 epochs (same as insurance_model_1)
insurance_model_4.fit(X_train_normal, y_train, epochs=200, verbose=0)

In [21]:
# Evaulate 4rd model
insurance_model_4_loss, insurance_model_4_mae = insurance_model_4.evaluate(X_test_normal, y_test)

9/9 [==============================] - 0s 638us/step - loss: 3821.8406 - mae: 3821.8406


# Model 5 --> RMS Prop with momemtum

In [22]:
# learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    #name='RMSprop'

    # Set random seed
tf.random.set_seed(42)

insurance_model_5 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),

  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
insurance_model_5.compile(loss=tf.keras.losses.mae, optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, momentum =0.9), metrics=['mae'])

# Fit the model for 200 epochs (same as insurance_model_1)
insurance_model_5.fit(X_train_normal, y_train, epochs=200, verbose=0)

 

In [23]:
# Evaulate 5rd model
insurance_model_5_loss, insurance_model_5_mae = insurance_model_5.evaluate(X_test_normal, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 3160.7859 - mae: 3160.7859


# Model 6 --> Adagrad + more epocs

In [25]:
    # Set random seed
tf.random.set_seed(42)


insurance_model_6 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),

  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
insurance_model_6.compile(loss=tf.keras.losses.mae, optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001), metrics=['mae'])

# Fit the model for 200 epochs (same as insurance_model_1)
insurance_model_6.fit(X_train_normal, y_train, epochs=200, verbose=0)

In [26]:
# Evaulate 6th model
insurance_model_6_loss, insurance_model_6_mae = insurance_model_6.evaluate(X_test_normal, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 12791.0166 - mae: 12791.0166


# Model 7 --> Nadam

In [27]:

    # Set random seed
tf.random.set_seed(42)


insurance_model_7 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),

  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
insurance_model_7.compile(loss=tf.keras.losses.mae, optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001), metrics=['mae'])

# Fit the model for 200 epochs (same as insurance_model_1)
insurance_model_7.fit(X_train_normal, y_train, epochs=200, verbose=0)



In [28]:
# Evaulate 7th model
insurance_model_7_loss, insurance_model_7_mae = insurance_model_7.evaluate(X_test_normal, y_test)

9/9 [==============================] - 0s 3ms/step - loss: 3169.7285 - mae: 3169.7285
